## ChatPromptTemplate (LangChain)


In **LangChain**, a **ChatPromptTemplate** is a **structured prompt abstraction** designed for **chat-based LLMs**.

> It converts structured inputs into a **sequence of role-based messages** (`system`, `human`, `assistant`) that are sent to a ChatModel.

This is the **primary and recommended prompt type** for all modern LangChain systems.

---

### Why ChatPromptTemplate Exists

Chat models require:

* Role separation
* Instruction hierarchy
* Tool awareness
* Memory injection

Raw strings cannot express this reliably.

ChatPromptTemplate:

* Preserves message roles
* Enables agents and tool calling
* Supports memory and RAG
* Composes cleanly with LCEL

---

### ChatPromptTemplate vs PromptTemplate

#### Key Difference

| Aspect       | PromptTemplate | ChatPromptTemplate  |
| ------------ | -------------- | ------------------- |
| Input        | Plain text     | Structured messages |
| Output       | String         | List of messages    |
| Agents       | ❌              | ✅                   |
| Tool calling | ❌              | ✅                   |
| Memory       | ❌              | ✅                   |
| RAG          | Limited        | Native              |

---

### Basic ChatPromptTemplate Demonstration

#### Creating a ChatPromptTemplate



In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])


### Formatting Messages


In [14]:

messages = prompt.format_messages(
    input="What is LangChain?"
)

for msg in messages:
    print(msg)


content='You are a helpful assistant.' additional_kwargs={} response_metadata={}
content='What is LangChain?' additional_kwargs={} response_metadata={}




Output:

```
SystemMessage(...)
HumanMessage(...)
```

---

### ChatPromptTemplate with Chat Model



In [15]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | llm

response = chain.invoke({"input": "Explain RAG"})

print(response.content)


RAG stands for Retrieve and Generate, which is a framework commonly used in natural language processing (NLP) for tasks such as question answering and dialogue generation. It combines two key capabilities of AI models: retrieval and generation.

1. **Retrieve**: This component involves searching through a large database or corpus of text to find relevant information that can answer a specific question or provide context for a conversation. This is often done using techniques like vector similarity, where queries are compared to a set of documents to find the most relevant matches.

2. **Generate**: After retrieving the relevant information, the model generates a response based on the retrieved data. This is typically done using a generative model, such as a sequence-to-sequence model or transformer architecture, which can formulate coherent and contextually appropriate responses.

### Advantages of the RAG Framework:
- **Improved Accuracy**: By combining retrieval and generation, RAG c


---

### Variables and Validation

#### Declaring Variables Implicitly

```python
("human", "{question}")
```

LangChain automatically:

* Detects required variables
* Raises error if missing

---

### Partial Variables (Static Instructions)



In [16]:
prompt = ChatPromptTemplate(
    messages=[
        ("system", "{role}"),
        ("human", "{question}")
    ],
    partial_variables={
        "role": "You are an IT support assistant."
    }
)

chain = prompt | llm

response = chain.invoke({"question": "Explain RAG"})

print(response.content)


RAG stands for "Red, Amber, Green," and it is a color-coding system often used for project management, reporting, and performance monitoring. Each color represents a status or level of progress:

1. **Red**: Indicates significant issues that need immediate attention. This status suggests that a project or task is off track, facing serious risks, or not meeting its objectives.

2. **Amber** (or Yellow): Represents caution. This status indicates that while the project or task is not currently in jeopardy, there are potential issues or concerns that could escalate if not addressed. It may require monitoring or some corrective actions.

3. **Green**: Signifies that everything is on track. The project or task is proceeding as planned, meeting its deadlines, and objectives are being achieved.

The RAG status is often presented in the form of dashboards or reports to provide a quick visual reference for stakeholders to understand the health of a project or initiative. It allows for easier dec


---

### MessagesPlaceholder (Critical Concept)

#### Injecting Memory or Scratchpad



In [17]:

from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])




Used for:

* Conversation memory
* Agent scratchpads
* Tool call history

---

### ChatPromptTemplate in Agents

#### Required Agent Parameters

For OpenAI tools agents:

* `{input}` – user query
* `{agent_scratchpad}` – tool call history

```python
prompt = ChatPromptTemplate.from_messages([
    ("system", "Use tools if needed."),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}")
])
```

⚠️ `agent_scratchpad` must be a MessagesPlaceholder or assistant slot.

---

### ChatPromptTemplate in RAG

#### Context Injection



In [18]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer using the context only."),
    ("human", "Context:\n{context}\n\nQuestion:\n{input}")
])

prompt.format(
    context="LangChain is a framework for building applications with LLMs.",
    input="What is LangChain?"
)

'System: Answer using the context only.\nHuman: Context:\nLangChain is a framework for building applications with LLMs.\n\nQuestion:\nWhat is LangChain?'



Retriever output populates `{context}`.

---

### ChatPromptTemplate with Structured Output



In [19]:
from pydantic import BaseModel

class Ticket(BaseModel):
    category: str
    priority: str

structured_llm = llm.with_structured_output(Ticket)

chain = prompt | structured_llm

result = chain.invoke({"input": "Email service is down", "context": "The email service is experiencing an outage affecting multiple users."})
print(result)


category='Service Outage' priority='High'



---

### Advanced Composition with LCEL



In [20]:
chain = (
    {
        "input": lambda x: x["question"]
    }
    | prompt
    | llm
)




ChatPromptTemplate behaves as a Runnable.

---

### Common Mistakes

#### Using PromptTemplate for chat models

❌ Breaks agents and tools

#### Missing required variables

❌ Runtime KeyError

#### Hard-coding chat history

❌ Breaks memory

#### Referencing internal state directly

❌ Agent failures

---

### Best Practices

* Always use ChatPromptTemplate for new systems
* Keep system prompts concise
* Inject dynamic content via placeholders
* Never expose chain-of-thought
* Version prompts like code

---

### When NOT to Use ChatPromptTemplate

* Legacy text-only models
* Simple batch text generation
* Non-chat inference pipelines

---

### Interview-Ready Summary

> “ChatPromptTemplate is LangChain’s primary prompt abstraction for chat-based LLMs. It produces structured, role-based messages, enabling agents, tools, memory, and RAG pipelines while enforcing input validation and composability.”

---

### Rule of Thumb

* **Chat / RAG / Agents → ChatPromptTemplate**
* **Text-only → PromptTemplate**
* **Production → Always structured prompts**
